In [1]:
from google.colab import files
files.upload()

Saving ecommerce_dataset.csv to ecommerce_dataset.csv


{'ecommerce_dataset.csv': b'\xef\xbb\xbfID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N\r\n1,D,Flight,4,2,177,3,low,F,44,1233,1\r\n2,F,Flight,4,5,216,2,low,M,59,3088,1\r\n3,A,Flight,2,2,183,4,low,M,48,3374,1\r\n4,B,Flight,3,3,176,4,medium,M,10,1177,1\r\n5,C,Flight,2,2,184,3,medium,F,46,2484,1\r\n6,F,Flight,3,1,162,3,medium,F,12,1417,1\r\n7,D,Flight,3,4,250,3,low,F,3,2371,1\r\n8,F,Flight,4,1,233,2,low,F,48,2804,1\r\n9,A,Flight,3,4,150,3,low,F,11,1861,1\r\n10,B,Flight,3,2,164,3,medium,F,29,1187,1\r\n11,C,Flight,3,4,189,2,medium,M,12,2888,1\r\n12,F,Flight,4,5,232,3,medium,F,32,3253,1\r\n13,D,Flight,3,5,198,3,medium,F,1,3667,1\r\n14,F,Flight,4,4,275,3,high,M,29,2602,1\r\n15,A,Flight,4,3,152,3,low,M,43,1009,1\r\n16,B,Flight,4,3,227,3,low,F,45,2707,1\r\n17,C,Flight,3,4,143,2,medium,F,6,1194,1\r\n18,F,Ship,5,5,227,3,medium,M,36,3952,1\r\n19,D,Ship,5,5,239,3,

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, classification_report
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm


##Problem Statement
In this homework, I would like to choose the "shopping" topic to predict whether an e-commerce shipment will arrive on time or not using ecommerce_dataset.csv.

##Data Collection and Preprocessing

In [41]:
df = pd.read_csv('/content/ecommerce_dataset.csv', ',')
df

<ipython-input-41-35171a2af3f6>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('/content/ecommerce_dataset.csv', ',')


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [43]:
df.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729,0.596691
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251,0.490584
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000,0.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000,0.000000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000,1.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000,1.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000,1.000000


In [44]:
df.isnull().any()

ID                     False
Warehouse_block        False
Mode_of_Shipment       False
Customer_care_calls    False
Customer_rating        False
Cost_of_the_Product    False
Prior_purchases        False
Product_importance     False
Gender                 False
Discount_offered       False
Weight_in_gms          False
Reached.on.Time_Y.N    False
dtype: bool

There are not missing values in the dataset, so we don't have to impute or drop missing values.

In [45]:
#Drop irrelavent feature ID
df = df.drop(['ID'], axis=1)
#One-hot encoding
categorical_variables = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
numerical_variables = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',	'Discount_offered', 'Weight_in_gms']
data = pd.get_dummies(df, columns=categorical_variables)

data.rename(columns = {'Reached.on.Time_Y.N': 'On_time'}, inplace = True)
data = data.drop(['Gender_F'], axis=1)
data

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,On_time,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_M
0,4,2,177,3,44,1233,1,0,0,0,1,0,1,0,0,0,1,0,0
1,4,5,216,2,59,3088,1,0,0,0,0,1,1,0,0,0,1,0,1
2,2,2,183,4,48,3374,1,1,0,0,0,0,1,0,0,0,1,0,1
3,3,3,176,4,10,1177,1,0,1,0,0,0,1,0,0,0,0,1,1
4,2,2,184,3,46,2484,1,0,0,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,4,1,252,5,1,1538,1,1,0,0,0,0,0,0,1,0,0,1,0
10995,4,1,232,5,6,1247,0,0,1,0,0,0,0,0,1,0,0,1,0
10996,5,4,242,5,4,1155,0,0,0,1,0,0,0,0,1,0,1,0,0
10997,5,2,223,6,2,1210,0,0,0,0,0,1,0,0,1,0,0,1,1


In [46]:
#Split data to training and test sets
target = data['On_time'].to_numpy()
data = data.drop(['On_time'], axis=1)
columns = data.columns
drop_index = [columns.get_loc(c) for c in['Warehouse_block_A', 'Mode_of_Shipment_Flight', 'Product_importance_high']]
data = data.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [47]:
X_train[0,:]

array([   3,    5,  178,    3,    7, 4135,    0,    0,    0,    0,    1,
          0,    0,    1,    0,    1,    0,    0])

In [48]:
#Normalize data with z-score standardization
scaler = StandardScaler()

# Fit the scaler on the training data and standardize the first 5 numerical columns
scaler.fit(X_train[:,:6])

X_train_num = scaler.transform(X_train[:,:6])
X_test_num = scaler.transform(X_test[:,:6])

X_train = np.concatenate([X_train_num, X_train[:, 6:]], axis=1)
X_test = np.concatenate([X_test_num, X_test[:, 6:]], axis=1)


In [49]:
X_train[0,:]

array([-0.92560498,  1.42208705, -0.67011986, -0.3786802 , -0.39291128,
        0.31190127,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ])

In [50]:
X_train_num

array([[-0.92560498,  1.42208705, -0.67011986, -0.3786802 , -0.39291128,
         0.31190127],
       [ 1.69710806,  0.00394867, -0.17324572,  0.27363129, -0.63896845,
         0.53260565],
       [-1.79984266,  1.42208705,  1.37948593, -0.3786802 , -0.45442557,
        -1.58517822],
       ...,
       [-0.0513673 , -0.70512051, -1.5189465 , -1.03099169, -0.70048274,
         1.33227692],
       [ 0.82287038,  0.00394867, -1.37402487,  1.57825427,  1.32948894,
        -1.59923972],
       [-0.0513673 ,  0.00394867, -1.1462909 , -0.3786802 , -0.70048274,
         0.55583769]])

##Modeling

In [51]:
models = []
results = pd.DataFrame({
    'Accuracy': [],
    'AUC': [],
    'F1_Score': [],
    'Precision': [],
    'Recall': []
})
results

,Accuracy,AUC,F1_Score,Precision,Recall


**KNN**

In [52]:
def euclidean_distance(p1, p2):
    # calculate the Euclidean distance between two vectors
    distance = np.sqrt(np.sum((p1 - p2)**2))
    return distance

def most_common(List):
    #find the most common
    return max(set(List), key = List.count)

def knnClassifier(X_train, X_test, y_train, k):
    y_pred = []
    for x_test in tqdm(X_test):
        #Compute distance between x_test for each x_train
        distances = [euclidean_distance(x_test, x_train) for x_train in X_train]
        #Get k nearest neighbors
        sort = sorted(zip(distances, y_train))[:k]
        y_nearest_neighbors = [y for _, y in sort]

        y_pred.append(most_common(y_nearest_neighbors))

    return np.array(y_pred)


In [53]:
y_pred = knnClassifier(X_train, X_test, y_train, 4)

100%|██████████| 2200/2200 [03:09<00:00, 11.62it/s]


In [54]:
y_pred

array([1, 0, 1, ..., 0, 0, 0])

In [55]:
def metrics_evaluation(y_test, y_pred, model):
    metrics_list = []
    accuracy = metrics.accuracy_score(y_test, y_pred)
    metrics_list.append(accuracy)

    fpr, tpr, _ = metrics.roc_curve(y_true=y_test, y_score=y_pred)

    auc_pred = metrics.auc(x=fpr, y=tpr)
    metrics_list.append(auc_pred)

    f1_pred = metrics.f1_score(y_test, y_pred)
    metrics_list.append(f1_pred)

    precision_pred = metrics.precision_score(y_test, y_pred)
    metrics_list.append(precision_pred)

    recall_pred = metrics.recall_score(y_test, y_pred)
    metrics_list.append(recall_pred)

    results.loc[model] = metrics_list
    print(f'Accuracy is {np.round(accuracy,4)}')
    print(f'AUC is {np.round(auc_pred,4)}')
    print(f'F1 score is {np.round(f1_pred,4)}')
    print(f'Precision is {np.round(precision_pred,4)}')
    print(f'Recall is {np.round(recall_pred,4)}')

In [56]:
metrics_evaluation(y_test, y_pred, 'KNN')

Accuracy is 0.6445
AUC is 0.6635
F1 score is 0.6521
Precision is 0.7773
Recall is 0.5617


**SVM**

In [57]:
#delete columns to avoid multicollinearity
X_train_drop = np.delete(X_train, drop_index, 1)
X_test_drop = np.delete(X_test, drop_index, 1)

In [58]:
svm = LinearSVC(C=0.01,dual = False).fit(X_train_drop, y_train)
y_pred = svm.predict(X_test_drop)

In [59]:
metrics_evaluation(y_test, y_pred, 'SVM')

Accuracy is 0.6409
AUC is 0.6313
F1 score is 0.6928
Precision is 0.7032
Recall is 0.6828


**Decision Tree**

In [60]:
decision_tree_model = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

In [61]:
dt_importance = decision_tree_model.feature_importances_

In [62]:
metrics_evaluation(y_test, y_pred, 'Decision Tree')

Accuracy is 0.6477
AUC is 0.6348
F1 score is 0.7034
Precision is 0.7026
Recall is 0.7042


**Random Forest**

In [63]:
random_forest_model = RandomForestClassifier().fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)

In [64]:
rf_importance = random_forest_model.feature_importances_

In [65]:
metrics_evaluation(y_test, y_pred, 'Random Forest')

Accuracy is 0.6609
AUC is 0.6633
F1 score is 0.6948
Precision is 0.7454
Recall is 0.6506


**Gradient Boost**

In [66]:
gradient_boost_model = GradientBoostingClassifier().fit(X_train, y_train)
y_pred = gradient_boost_model.predict(X_test)

In [67]:
gb_importance = gradient_boost_model.feature_importances_

In [68]:
metrics_evaluation(y_test, y_pred, 'Gradient Boost')

Accuracy is 0.6882
AUC is 0.7159
F1 score is 0.6833
Precision is 0.8595
Recall is 0.567


##Hyperparameter Tuning

In [69]:
from sklearn.model_selection import GridSearchCV

Decision Tree

In [70]:
# Define the hyperparameters values
max_depth_values = [3, 5, 10, None]
min_samples_split_values = [2, 5, 10]

best_accuracy_dt = 0
best_params_dt = {}

for max_depth in max_depth_values:
    for min_samples_split in min_samples_split_values:
        dt = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, random_state=42).fit(X_train, y_train)
        y_pred = dt.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        fpr, tpr, _ = metrics.roc_curve(y_true=y_test, y_score=y_pred)
        auc_pred = metrics.auc(x=fpr, y=tpr)
        precision_pred = metrics.precision_score(y_test, y_pred)
        recall_pred = metrics.recall_score(y_test, y_pred)
        f1_pred = metrics.f1_score(y_test, y_pred)

        if accuracy > best_accuracy_dt:
            best_accuracy_dt = accuracy
            best_pred = y_pred
            best_params_dt = {'max_depth': max_depth, 'min_samples_split': min_samples_split}

print("Best Parameters for Decision Tree:", best_params_dt)
print("Best Accuracy for Decision Tree:", best_accuracy_dt)


Best Parameters for Decision Tree: {'max_depth': 5, 'min_samples_split': 2}
Best Accuracy for Decision Tree: 0.685


In [71]:
metrics_evaluation(y_test, best_pred, 'Decision Tree (Hyperparameter Tuning)')

Accuracy is 0.685
AUC is 0.7225
F1 score is 0.6624
Precision is 0.9091
Recall is 0.5211


Gradient Boost

In [72]:
# Define the hyperparameters values
n_estimators_values = [100, 200, 300]
learning_rate_values = [0.01, 0.1, 0.2]

best_accuracy_gb = 0
best_params_gb = {}

for n_estimators in n_estimators_values:
    for learning_rate in learning_rate_values:
        gb = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42).fit(X_train, y_train)
        y_pred = gb.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        fpr, tpr, _ = metrics.roc_curve(y_true=y_test, y_score=y_pred)
        auc_pred = metrics.auc(x=fpr, y=tpr)
        precision_pred = metrics.precision_score(y_test, y_pred)
        recall_pred = metrics.recall_score(y_test, y_pred)
        f1_pred = metrics.f1_score(y_test, y_pred)

        if accuracy > best_accuracy_gb:
            best_accuracy_gb = accuracy
            best_pred = y_pred
            best_params_gb = {'n_estimators': n_estimators, 'learning_rate': learning_rate}

print("Best Parameters for Gradient Boost:", best_params_gb)
print("Best Accuracy for Gradient Boost:", best_accuracy_gb)

Best Parameters for Gradient Boost: {'n_estimators': 300, 'learning_rate': 0.01}
Best Accuracy for Gradient Boost: 0.6913636363636364


In [73]:
metrics_evaluation(y_test, best_pred, 'Gradient Boost (Hyperparameter Tuning)')

Accuracy is 0.6914
AUC is 0.7316
F1 score is 0.6647
Precision is 0.9347
Recall is 0.5157


##Results

In [74]:
results

,Accuracy,AUC,F1_Score,Precision,Recall
KNN,0.644545,0.663524,0.652135,0.777306,0.561686
SVM,0.640909,0.631323,0.692846,0.703236,0.682759
Decision Tree,0.647727,0.634789,0.703406,0.702599,0.704215
Random Forest,0.660909,0.663276,0.694763,0.745391,0.650575
Gradient Boost,0.688182,0.715927,0.683287,0.859466,0.567050
Decision Tree (Hyperparameter Tuning),0.685000,0.722548,0.662445,0.909091,0.521073
Gradient Boost (Hyperparameter Tuning),0.691364,0.731597,0.664691,0.934722,0.515709


In [75]:
importances_features = pd.DataFrame({
    'Feature': columns,
    'Decision Tree': dt_importance,
    'Random Forest': rf_importance,
    'Gradient Boosting': gb_importance
})
importances_features

,Feature,Decision Tree,Random Forest,Gradient Boosting
0,Customer_care_calls,0.044825,0.056068,0.008658
1,Customer_rating,0.045051,0.058905,0.004326
2,Cost_of_the_Product,0.163947,0.171740,0.044712
3,Prior_purchases,0.055980,0.061099,0.046164
4,Discount_offered,0.301634,0.229097,0.740991
5,Weight_in_gms,0.216840,0.262890,0.143183
6,Warehouse_block_A,0.020179,0.011886,0.000954
7,Warehouse_block_B,0.011781,0.013257,0.001359
8,Warehouse_block_C,0.012821,0.012728,0.000808
9,Warehouse_block_D,0.017008,0.012715,0.002545
